# CIFAR-10 Challange - 김영인

1. resnet18
2. \+ Data Augmentation(RandomResizedCrop, RandomHorizontalFlip)
3. \+ Data Normalize
4. \+ regularization(dropout, weight decay)
5. \+ Optimizer(momentum)

In [1]:
import random

import torch
import numpy as np
from matplotlib import pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets

random_seed = 4332
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

device0 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device1 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

device = device0
print(f"device: {device}") if torch.cuda.is_available() else print("device: cpu")

device: cuda:0


## Hyper parameter Setting 

In [2]:
learning_rate = 1e-2
training_epochs = 50
batch_size = 64
momentum = 0.9
weight_decay = 5e-4
dropout_rate = .4
scheduler_step = 10
scheduler_gamma = 0.7

## Load & Preprocess Data

In [3]:
transform_train = transforms.Compose(
    [transforms.RandomResizedCrop(224), # data augmentation, 224: image size, ImageNet pretrained model에 맞추기 위해서 224 size로 설정
     transforms.RandomHorizontalFlip(), # data augmentation, 좌우로 대칭
     transforms.ToTensor(), # numpy array를 pytorch tensor로 바꿔주는 역할
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # dataset의 mean, std를 이용해서 -1~1 로 normalize
    ])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Load and Define pretrained model and fine-tuning

In [4]:
class ResNet18(nn.Module):
    def __init__ (self):
        super(ResNet18, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(1000, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 10)
        )
        
    def forward(self, x):
        x = self.resnet(x) # batch_size * 10
        x = self.classifier(x)
        return x

## Load Model

In [5]:
net = ResNet18()
net = net.to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                      step_size = scheduler_step,
                                      gamma = scheduler_gamma)

## Train

In [7]:
net.train()

for epoch in range(training_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = net(inputs) 
        loss =  criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            
        if i == 500:
            class_correct = np.zeros(10)
            class_total = np.zeros(10)

            net.eval()

            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = net(images)
                    _, predicted = torch.max(outputs, 1)
                    c = (predicted == labels).squeeze()
                    for i in range(4):
                        label = labels[i]
                        class_correct[label] += c[i].item()
                        class_total[label] += 1
            print('==================')
            print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))
            print('==================')
            
            net.train()
        
print('Finished Training')

[1,   100] loss: 1.511
[1,   200] loss: 1.079
[1,   300] loss: 0.974
[1,   400] loss: 0.898
[1,   500] loss: 0.875
Mean Accuracy :  72.66
[1,   600] loss: 0.798
[1,   700] loss: 0.815
[2,   100] loss: 0.741
[2,   200] loss: 0.746
[2,   300] loss: 0.732
[2,   400] loss: 0.696
[2,   500] loss: 0.711
Mean Accuracy :  75.58
[2,   600] loss: 0.713
[2,   700] loss: 0.720
[3,   100] loss: 0.684
[3,   200] loss: 0.663
[3,   300] loss: 0.658
[3,   400] loss: 0.683
[3,   500] loss: 0.627
Mean Accuracy :  78.49
[3,   600] loss: 0.667
[3,   700] loss: 0.680
[4,   100] loss: 0.609
[4,   200] loss: 0.632
[4,   300] loss: 0.614
[4,   400] loss: 0.603
[4,   500] loss: 0.610
Mean Accuracy :  78.81
[4,   600] loss: 0.624
[4,   700] loss: 0.595
[5,   100] loss: 0.584
[5,   200] loss: 0.591
[5,   300] loss: 0.608
[5,   400] loss: 0.635
[5,   500] loss: 0.591
Mean Accuracy :  80.15
[5,   600] loss: 0.591
[5,   700] loss: 0.599
[6,   100] loss: 0.588
[6,   200] loss: 0.566
[6,   300] loss: 0.575
[6,   400] 

[37,   200] loss: 0.492
[37,   300] loss: 0.474
[37,   400] loss: 0.481
[37,   500] loss: 0.463
Mean Accuracy :  81.16
[37,   600] loss: 0.477
[37,   700] loss: 0.490
[38,   100] loss: 0.500
[38,   200] loss: 0.462
[38,   300] loss: 0.483
[38,   400] loss: 0.484
[38,   500] loss: 0.493
Mean Accuracy :  81.02
[38,   600] loss: 0.487
[38,   700] loss: 0.470
[39,   100] loss: 0.452
[39,   200] loss: 0.487
[39,   300] loss: 0.476
[39,   400] loss: 0.473
[39,   500] loss: 0.482
Mean Accuracy :  81.76
[39,   600] loss: 0.480
[39,   700] loss: 0.474
[40,   100] loss: 0.460
[40,   200] loss: 0.462
[40,   300] loss: 0.464
[40,   400] loss: 0.473
[40,   500] loss: 0.466
Mean Accuracy :  81.88
[40,   600] loss: 0.496
[40,   700] loss: 0.462
[41,   100] loss: 0.481
[41,   200] loss: 0.461
[41,   300] loss: 0.473
[41,   400] loss: 0.472
[41,   500] loss: 0.480
Mean Accuracy :  79.27
[41,   600] loss: 0.488
[41,   700] loss: 0.472
[42,   100] loss: 0.477
[42,   200] loss: 0.460
[42,   300] loss: 0.4

## Test

In [9]:
class_correct = np.zeros(10)
class_total = np.zeros(10)

net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))

Accuracy of plane : 86.900000 %
Accuracy of   car : 88.700000 %
Accuracy of  bird : 74.900000 %
Accuracy of   cat : 77.600000 %
Accuracy of  deer : 79.900000 %
Accuracy of   dog : 79.200000 %
Accuracy of  frog : 86.800000 %
Accuracy of horse : 80.900000 %
Accuracy of  ship : 86.200000 %
Accuracy of truck : 82.400000 %
Mean Accuracy :  82.35
